# Finance SQL Views Installer

This notebook fetches the latest SQL definition from GitHub and executes it against the specified SQL Analytics Endpoint.

**GitHub Source:** [finance-sql-views.sql](https://github.com/navida-Informationssysteme/BC2Fabric-Toolbox/blob/main/Accelerators/Finance-Reporting/finance-sql-views.sql)

In [ ]:
import requests
import pyodbc
import struct
import re

# Try importing the modern notebookutils, fallback to mssparkutils if needed
try:
    from notebookutils import credentials
except ImportError:
    from mssparkutils import credentials

# -------------------------------------------------------
# Configuration
# -------------------------------------------------------
# URL to the raw SQL file on GitHub
GITHUB_RAW_URL = "https://raw.githubusercontent.com/navida-Informationssysteme/BC2Fabric-Toolbox/main/Accelerators/Finance-Reporting/finance-sql-views.sql"

# SQL Endpoint and Database details
SQL_ENDPOINT = "ryxn3t66mqleblwe5fkgsvj3di-slpu2ietzspuxpjlkbn77ew3ua.datawarehouse.fabric.microsoft.com"
DATABASE_NAME = "bc2fabric_mirror"

# -------------------------------------------------------
# 1. Fetch SQL Content
# -------------------------------------------------------
print(f"Fetching SQL from: {GITHUB_RAW_URL}")
try:
    response = requests.get(GITHUB_RAW_URL)
    response.raise_for_status()
    sql_content = response.text
    print("SQL content fetched successfully.")
except Exception as e:
    print(f"Failed to fetch SQL file: {e}")
    raise

# -------------------------------------------------------
# 2. Connect to SQL Endpoint
# -------------------------------------------------------
# Get Entra ID (AAD) token
token = credentials.getToken("pbi")
token_bytes = token.encode("UTF-16LE")
token_struct = struct.pack(f'<I{len(token_bytes)}s', len(token_bytes), token_bytes)
SQL_COPT_SS_ACCESS_TOKEN = 1256 

# Construct Connection String
conn_str = (
    f"Driver={{ODBC Driver 18 for SQL Server}};"
    f"Server={SQL_ENDPOINT},1433;"
    f"Database={DATABASE_NAME};"
    f"Encrypt=yes;"
    f"TrustServerCertificate=no;"
    f"Connection Timeout=30;"
)

print(f"Connecting to {DATABASE_NAME}...")

try:
    # Autocommit is often required for DDL statements in loops
    with pyodbc.connect(conn_str, attrs_before={SQL_COPT_SS_ACCESS_TOKEN: token_struct}, autocommit=True) as conn:
        cursor = conn.cursor()
        
        # -------------------------------------------------------
        # 3. Robust SQL Parsing
        # -------------------------------------------------------
        # Handles "GO", "GO;", case insensitivity, and surrounding whitespace
        split_pattern = r'^\s*GO\s*;?\s*$'
        
        statements = re.split(split_pattern, sql_content, flags=re.IGNORECASE | re.MULTILINE)
        
        # Filter out empty strings
        statements = [stmt.strip() for stmt in statements if stmt.strip()]
        
        print(f"Found {len(statements)} primary batches to execute.")
        
        for i, stmt in enumerate(statements):
            # -------------------------------------------------------
            # 4. Sub-batch Splitting Logic (Schema + View Fix)
            # -------------------------------------------------------
            # T-SQL requires CREATE VIEW to be the first statement in a batch.
            # If a file has "CREATE SCHEMA ... CREATE VIEW" without a GO, we split manually.
            sub_statements = [stmt]
            
            if "CREATE SCHEMA" in stmt.upper() and "CREATE OR ALTER VIEW" in stmt.upper():
                 # Lookahead split: Split right before "CREATE OR ALTER VIEW"
                 sub_statements = re.split(r'(?=CREATE\s+OR\s+ALTER\s+VIEW)', stmt, flags=re.IGNORECASE)

            for sub_stmt in sub_statements:
                clean_stmt = sub_stmt.strip()
                if not clean_stmt: continue
                
                print(f"Executing batch {i+1} part...")
                try:
                    cursor.execute(clean_stmt)
                except pyodbc.Error as e:
                    print(f"Error executing batch {i+1}:")
                    print(f"Query start: {clean_stmt[:200]}...") 
                    print(f"Error details: {e}")
                    # Raising error ensures we don't leave the DB in a half-baked state silently
                    raise 

        print("All views created successfully.")
        
except Exception as e:
    print("An error occurred during the database operation.")
    print(e)